In [2]:
import pandas as pd
import numpy as np

In [3]:
# Importamos la data de las plataformas presentadas en la Datasets y agregamos la columna Id_Plataform para identificar 
# cada plataforma
data1 = pd.read_csv("./Datasets/amazon_prime_titles.csv",encoding="utf-8")
data1.insert(loc=1, column="Id_Plataform", value=1)

data2 = pd.read_csv("./Datasets/disney_plus_titles.csv",encoding="utf-8")
data2.insert(loc=1, column="Id_Plataform", value=2)

data3 = pd.read_csv("./Datasets/hulu_titles.csv",encoding="utf-8")
data3.insert(loc=1, column="Id_Plataform", value=3)

data4 = pd.read_json("./Datasets/netflix_titles.json",encoding="utf-8")
data4.insert(loc=1, column="Id_Plataform", value=4)

In [4]:
# Unimos la data en una sola tabla y además eliminamos las columnas que no serán de utilidad para las 
# consignas indicadas
df=pd.concat([data1,data2,data3,data4],ignore_index=True)
df.drop(columns=["show_id","director","date_added","rating","description","country"], axis = 1, inplace=True)

In [5]:
# Reemplazamos los valores nan
df.replace(np.nan,"",inplace=True)


In [6]:
# Generamos una nueva tabla Id_Producción para hacer único los títulos, por tipo y año de estreno
df["Id_Producción"]=df["release_year"].apply(str)+"-"+df["title"]+"-"+df["type"]

In [7]:
# Contabilizamos los datos vacios
df[df==""].count()

Id_Plataform        0
type                0
title               0
cast             5321
release_year        0
duration          482
listed_in           0
Id_Producción       0
dtype: int64

In [8]:
# Reemplazamos los datos vacios en cast por algun duplicado con el mismo Id_Producción
for i in df[df["cast"]==""].index: #df[df["cast"]==""].index nos da una lista con indices de vacio para la columna cast
    for j in df[df["Id_Producción"].duplicated(False)].index:#df[df["Id_Producción"].duplicated(False)].index nos da una lista con indices duplicados sin excluir los primeros encontrados.
        if i!=j and  df.loc[j,"cast"]!="" and df.loc[i,"Id_Producción"]==df.loc[j,"Id_Producción"]:
            df.loc[i,"cast"]=df.loc[j,"cast"]
            break

In [9]:
# Reemplazamos los datos vacios en duration por algun duplicado con el mismo Id_Producción
for i in df[df["duration"]==""].index: #df[df["duration"]==""].index nos da una lista con indices de vacio para la columna cast
    for j in df[df["Id_Producción"].duplicated(False)].index:#df[df["Id_Producción"].duplicated(False)].index nos da una lista con indices duplicados sin excluir los primeros encontrados.
        if i!=j and  df.loc[j,"duration"]!="" and df.loc[i,"Id_Producción"]==df.loc[j,"Id_Producción"]:
            df.loc[i,"duration"]=df.loc[j,"duration"]
            break

In [10]:
# Consultamos la cantidad de datos vacios nuevamente y verficamos una disminución de estos
df[df==""].count()

Id_Plataform        0
type                0
title               0
cast             5146
release_year        0
duration          451
listed_in           0
Id_Producción       0
dtype: int64

In [11]:
# Generamos la tabla dfcast atomizando los datos de la columna cast para la normalizacion F1
dfcast=df.loc[:,("Id_Plataform","Id_Producción","cast")]
a=[]
b=[]
c=[]
for i in range(0,len(dfcast.iloc[:,2])):
    if "," in dfcast.iloc[i,2]:
        for j in list(dfcast.iloc[i,2].split(",")):
            a.append(dfcast.iloc[i,0])
            b.append(dfcast.iloc[i,1])
            c.append(j)
    else:
        a.append(dfcast.iloc[i,0])
        b.append(dfcast.iloc[i,1])
        c.append(dfcast.iloc[i,2])

c=list(map(str.strip,c))
dfcast=pd.DataFrame({"Id_Plataform":a,
                "Id_Producción":b,
                "cast":c})

In [12]:
# Eliminamos los duplicados de la tabla dfcast
dfcast.drop_duplicates(inplace=True)
dfcast.reset_index(inplace=True,drop=True)
dfcast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120866 entries, 0 to 120865
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Id_Plataform   120866 non-null  int64 
 1   Id_Producción  120866 non-null  object
 2   cast           120866 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [17]:
# Generamos la tabla dflisted atomizando los datos de la columna listed_in para la normalizacion F1
dflisted=df.loc[:,("Id_Plataform","Id_Producción","listed_in")]
a=[]
b=[]
c=[]
for i in range(0,len(dflisted.iloc[:,2])):
    if "," in dflisted.iloc[i,2]:
        for j in list(dflisted.iloc[i,2].split(",")):
            a.append(dflisted.iloc[i,0])
            b.append(dflisted.iloc[i,1])
            c.append(j)
    else:
        a.append(dflisted.iloc[i,0])
        b.append(dflisted.iloc[i,1])
        c.append(dflisted.iloc[i,2])

c=list(map(str.strip,c))
dflisted=pd.DataFrame({"Id_Plataform":a,
                "Id_Producción":b,
                "listed_in":c})


In [18]:
# Eliminamos los duplicados de la tabla dflisted
dflisted.drop_duplicates(inplace=True)
dflisted.reset_index(inplace=True,drop=True)
dflisted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48303 entries, 0 to 48302
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id_Plataform   48303 non-null  int64 
 1   Id_Producción  48303 non-null  object
 2   listed_in      48303 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [19]:
# Generamos la tabla dfFilm
dfFilm=df.loc[:,("Id_Plataform","title","type","release_year","duration")]
dfFilm.drop_duplicates(inplace=True)
dfFilm.reset_index(inplace=True,drop=True)
dfFilm.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id_Plataform  22998 non-null  int64 
 1   title         22998 non-null  object
 2   type          22998 non-null  object
 3   release_year  22998 non-null  int64 
 4   duration      22998 non-null  object
dtypes: int64(2), object(3)
memory usage: 898.5+ KB


In [20]:
# Generamos la tabla DfPlataformas.
dfPlataformas=pd.DataFrame({"Id_Plataform":[1,2,3,4],
                            "Plataforma":["amazon","disney","hulu","netflix"]
                            })
dfPlataformas.head()

,Id_Plataform,Plataforma
0,1,amazon
1,2,disney
2,3,hulu
3,4,netflix


In [65]:
# Creamos los archivos csv de la data 
dfPlataformas.to_csv("Tablas/dfPlataformas.csv")
dfFilm.to_csv("Tablas/dfFilm.csv")
dfcast.to_csv("Tablas/dfcast.csv")
dflisted.to_csv("Tablas/dflisted.csv")

In [3]:
from fastapi import FastAPI
from starlette.responses import RedirectResponse
import pymysql

app = FastAPI()


def conectar_a_base_de_datos() :
    conexion = pymysql.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'mysql123M',
    db= 'pi',
    port= 3306
    )
    return conexion.cursor()


#1 Funcion que recibe año y plataforma y tipo de tiempo y retorna la pelicula o serie y su duracion maxima.
def max(anio,plataforma,time):
    cursor=conectar_a_base_de_datos()
    a=[]
    p=0
    anio=str(anio)
    if plataforma == "amazon":
        p = 1
    elif plataforma == "disney":
        p = 2
    elif plataforma == "hulu":
        p = 3
    elif plataforma == "netflix":
        p = 4
    else:
        "Inserte: amazon, disney, hulu o netflix"
    p=str(p) 
    if time=="min":
        query="select Id_Plataform, title, release_year, convert(SUBSTRING_INDEX(duration,' ', 1), UNSIGNED INTEGER)  AS tiempo, \
        SUBSTRING_INDEX(duration,' ', -1) as Unidades \
        from dffilm \
        where release_year = "+anio+" and Id_Plataform = '"+p+"' and SUBSTRING_INDEX(duration,' ', -1) like 'min%' \
        order by 4 desc \
        limit 1"
        cursor.execute(query)
    elif time=="season":
        query="select Id_Plataform, title, release_year, convert(SUBSTRING_INDEX(duration,' ', 1), UNSIGNED INTEGER)  AS tiempo, \
        SUBSTRING_INDEX(duration,' ', -1) as Unidades \
        from dffilm \
        where release_year = "+anio+" and Id_Plataform = '"+p+"' and SUBSTRING_INDEX(duration,' ', -1) like 'Season%' \
        order by 4 desc \
        limit 1"
        cursor.execute(query)
    else:
        return "Inserte [min] o [season] en el tercer parametro"

    for dato in cursor:
        a.append(dato)
    
    return a
def cant(plataforma):
    cursor=conectar_a_base_de_datos()
    a=[]
    p=0
    if plataforma == "amazon":
        p = 1
    elif plataforma == "disney":
        p = 2
    elif plataforma == "hulu":
        p = 3
    elif plataforma == "netflix":
        p = 4
    else:
        "Inserte: amazon, disney, hulu o netflix"
    p=str(p)
    cursor.execute("SELECT Id_PLataform, type ,count(*) as Cantidad \
    FROM dffilm \
    where Id_Plataform = '"+p+"' \
    group by Id_Plataform, type")
    for dato in cursor:
        a.append(dato)
    
    return a
def gen(genero):
    cursor=conectar_a_base_de_datos()
    a=[]
    query="SELECT Id_Plataform, listed_in, count(*) AS Cantidad\
    FROM dflisted \
    WHERE listed_in LIKE '"+genero+"' \
    GROUP BY Id_Plataform, listed_in \
    HAVING listed_in !='' \
    ORDER BY 3 DESC \
    LIMIT 1"
    cursor.execute(query)
    for dato in cursor:
        a.append(dato)
    
    return a
def act(plataforma,anio):
    cursor=conectar_a_base_de_datos()
    a=[]
    anio=str(anio)
    p=0
    if plataforma == "amazon":
        p = 1
    elif plataforma == "disney":
        p = 2
    elif plataforma == "hulu":
        p = 3
    elif plataforma == "netflix":
        p = 4
    else:
        "Inserte: amazon, disney, hulu o netflix"
    p=str(p)
    query="select Id_Plataform, cast, count(*) as Cantidad \
    from dfcast \
    where Id_plataform = '"+p+"' and Id_Producción like '"+anio+"%' \
    group by cast \
    having cast !='' \
    order by 3 desc \
    limit 1"
    cursor.execute(query)
    for dato in cursor:
        a.append(dato)
    
    return a

In [2]:
max(2018,"hulu","min")

[(3, 'The House That Jack Built', 2018, 151, 'min')]

In [4]:
cant("netflix")

[(4, 'Movie', 6131), (4, 'TV Show', 2676)]

In [14]:
gen("comedy")

[(1, 'Comedy', 2099)]

In [16]:
act("netflix",2018)

[(4, 'Andrea Libman', 8)]